In [2]:
# File: steam_crawler_test.py
import requests
import pandas as pd
import time
from tqdm import tqdm

def fetch_recent_reviews(appid, language, max_reviews):
    """爬取指定语言的最新评论（按时间倒序）"""
    url = f"https://store.steampowered.com/appreviews/{appid}"
    reviews = []
    cursor = '*'
    
    # 进度条配置
    pbar = tqdm(total=max_reviews, desc=f"爬取进度（{language}）", unit="条")
    
    while len(reviews) < max_reviews:
        params = {
            'json': 1,
            'filter': 'recent',
            'language': language,
            'num_per_page': 100,
            'cursor': cursor,
            'day_range': 9223372036854775807  # 最大时间范围
        }
        
        try:
            response = requests.get(url, params=params, timeout=15)
            response.raise_for_status()
            data = response.json()
            
            if not data.get('success', False):
                print(f"请求失败：{data.get('error', '未知错误')}")
                break
                
            batch = data.get('reviews', [])
            if not batch:
                break
                
            # 去重逻辑
            new_reviews = [r for r in batch if r['recommendationid'] not in {x['recommendationid'] for x in reviews}]
            reviews.extend(new_reviews)
            pbar.update(len(new_reviews))
            
            cursor = data.get('cursor')
            if not cursor or len(reviews) >= max_reviews:
                break
                
            time.sleep(1.5)  # 严格遵守反爬限制
            
        except Exception as e:
            print(f"发生错误: {str(e)}，重试中...")
            time.sleep(5)
    
    pbar.close()
    return pd.DataFrame(reviews[:max_reviews])

# 执行测试爬取
if __name__ == "__main__":
    appid = 2358720  # 黑神话：悟空
    df = fetch_recent_reviews(appid, 'english', 140000)
    
    # 保存完整字段
    df.to_csv('wukong_english_filterRecent_V2.csv', 
             index=False, 
             encoding='utf-8-sig',
             quoting=1)  # 处理含逗号的文本
    
    print(f"已保存 {len(df)} 条测试评论")


爬取进度（english）:   8%|▊         | 11298/140000 [04:55<54:31, 39.34条/s]


爬取进度（english）:  16%|█▌        | 22398/140000 [10:31<1:04:17, 30.49条/s]


爬取进度（english）:  24%|██▍       | 33498/140000 [16:26<54:52, 32.34条/s]


爬取进度（english）:  32%|███▏      | 44798/140000 [22:44<53:15, 29.79条/s]


爬取进度（english）:  40%|████      | 56094/140000 [29:27<50:50, 27.50条/s]


爬取进度（english）:  43%|████▎     | 60734/140000 [32:33<42:29, 31.09条/s]  


已保存 60734 条测试评论
